<a href="https://colab.research.google.com/github/raj-vijay/da/blob/master/04_Bin_Packing_Problem_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Bin Packing / Pandas

**BACKGROUND**

<p text-align = 'justify'>The bin packing problem is a direct generalisation of the knapsack problem. Instead of only one knapsack the bin packing problem tries to fit items into multiple bins of a given capacity while maximising the value. More formally, we are given 𝑛 items of size 𝑤1, … , 𝑤𝑛 and value 𝑣1, … , 𝑣𝑛that we want to distribute across 𝑚 bins of capacity 𝑀1, … , 𝑀𝑚.</p> 

<p text-align = 'justify'>If we use the Boolean decision variables 𝑥11, … , 𝑥𝑖𝑗, … , 𝑥𝑛𝑚 to indicate if an item 𝑖 is to be packed in bin 𝑗 we can formulate this problem as maximising the total value.</p>

<p align = 'center'>$V=\sum_{i, j} x_{i, j}v_{i}$</p>

Subject to the constraints that each item is only in one box:

<p align = 'center'>$\sum_{j} x_{i, j} \leq 1$

$i = 1, ..., n$</p>



And that the capacity of each box is not exceeded: 

<p align = 'center'>$\sum_{i} x_{i, j}w_{i} \leq M_{j}$

$j = 1, ..., m$</p>


In [ ]:
# Get the bin packing data
!wget https://github.com/raj-vijay/da/raw/master/files/bin_packing_data.xlsx

--2020-10-23 13:18:57--  https://github.com/raj-vijay/da/raw/master/files/bin_packing_data.xlsx
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/raj-vijay/da/master/files/bin_packing_data.xlsx [following]
--2020-10-23 13:18:57--  https://raw.githubusercontent.com/raj-vijay/da/master/files/bin_packing_data.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10885 (11K) [application/octet-stream]
Saving to: ‘bin_packing_data.xlsx’

bin_packing_data.xl 100%[===================>]  10.63K  --.-KB/s    in 0s      

2020-10-23 13:18:57 (100 MB/s) - ‘bin_packing_data.xlsx’ saved [10885/10885]



In [ ]:
!pip install ortools

     |████████████████████████████████| 13.7MB 261kB/s 
     |████████████████████████████████| 1.3MB 37.0MB/s 
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


In [ ]:
#Import Google OR tools libraries
from ortools.sat.python import cp_model
import pandas as pd

The Excel-file “bin_packing_data.xlsx” contains the input data, both a list of available containers as well as a list of items that need to be packed. 

Load the excel file and extract all relevant information.

In [ ]:
data = pd.read_excel("/content/bin_packing_data.xlsx", sheet_name=None)

In [ ]:
containers = data["Containers"]
items = data["Items"]

Create a CP-SAT model and add all necessary decision variables to that model.

In [ ]:
model = cp_model.CpModel()

Add the constraints to the model to ensure that each item can only go into one container.

In [ ]:
# one Boolean decision variable for every combination of item/bin
x = {}
for i in range(len(items)):
  for j in range(len(containers)):
    x[(i,j)] = model.NewBoolVar("x_%s_%s"%(i,j))

Add the constraints to the model to ensure that the container capacities are not exceeded.

In [ ]:
# every item is at most in one bin
for i in range(len(items)):
  model.Add(sum([x[(i,j)] for j in range(len(containers))])<=1)

Add the constraints to the model to ensure that the container capacities are not exceeded.

In [ ]:
# every bin's capacity is not exceeded
for j in range(len(containers)):
  model.Add(sum([items.iloc[i]["Weight"]*x[(i,j)] for i in range(len(items))])<=containers.iloc[j]["Maximum capacity"])

Add the objective function to the model and solve it using the CP-SAT solver.

In [ ]:
model.Maximize(sum([items.iloc[i]["Value"]*x[(i,j)] 
                        for i in range(len(items)) for j in range(len(containers))]))

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)   

In [ ]:
print(solver.StatusName(status))

OPTIMAL


How much value can be transported, both absolute and in percent of the total value of items?

In [ ]:
result_items = pd.DataFrame(index=items.index, columns=["Id", "Weight", "Value", "Container"])

In [ ]:
for i in range(len(items)):
  result_items["Id"][i] = items.iloc[i]["Id"]
  result_items["Weight"][i] = items.iloc[i]["Weight"]
  result_items["Value"][i] = items.iloc[i]["Value"]
  for j in range(len(containers)):
    if solver.Value(x[(i,j)]):
      result_items["Container"][i] = containers.iloc[j]["Id"]
      break

In [ ]:
result_containers = pd.DataFrame(index=containers.index, columns=["Id", "Maximum capacity", 
                                                                      "Weight (total)", 
                                                                      "Weight (percentage)", 
                                                                      "Value (total)", 
                                                                      "Value (percentage)"])

In [ ]:
for j in range(len(containers)):
  result_containers["Id"][j] = containers.iloc[j]["Id"]
  result_containers["Maximum capacity"][j] = containers.iloc[j]["Maximum capacity"]
  weight = 0
  value = 0
  for i in range(len(items)):
    if solver.Value(x[(i,j)]):
      weight += items.iloc[i]["Weight"]
      value += items.iloc[i]["Value"]
  result_containers["Weight (total)"][j] = weight
  result_containers["Weight (percentage)"][j] = 100*weight/containers.iloc[j]["Maximum capacity"]
  result_containers["Weight (total)"][j] = weight
  result_containers["Value (total)"][j] = value
  result_containers["Value (percentage)"][j] = 100*value/solver.BestObjectiveBound()

In [ ]:
writer = pd.ExcelWriter("Results.xlsx")
result_items.to_excel(writer, sheet_name="Items", index=False)
result_containers.to_excel(writer, sheet_name="Containers", index=False)
writer.close()

In [ ]:
print("Total item value", sum(items["Value"]))
print("Total packed value", solver.BestObjectiveBound())
print("Percentage", round(100*solver.BestObjectiveBound()/sum(items["Value"]),2))

Total item value 4420
Total packed value 2500.0
Percentage 56.56
